In [ ]:
%pwd


'/content'

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, RidgeClassifier
import time as tm
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

In [2]:
# Assuming the file has numeric data separated by spaces
X = np.loadtxt("public_trn.txt")
X_test = np.loadtxt("public_tst.txt")

In [ ]:

# def compute_tu_k(c):
#     k = len(c)
#     tu_k = np.zeros(17*k+15)

#     for i in range(k):
#         tu_k[i] = 1 - 2*c[i]

#     x_k = np.zeros(k-1)
#     for i in range(k-1):
#       prod = 1
#       for j in range(i,k):
#         prod *= (1 - 2*c[j])
#       x_k[i] = prod
#     count = k
#     for i in range(1, k):
#         prod = 1 - 2*c[i]
#         for j in range(i):
#             tu_k[count] = prod*x_k[j]
#             count += 1

#     for i in range(16*k+16,17*k+15):
#         tu_k[i] = x_k[i-16*k-16]

#     return tu_k

# def my_map(challenges):
#     n_samples = challenges.shape[0]
#     k = challenges.shape[1]
#     transformed_challenges = np.zeros((n_samples, 17*k+15))

#     for i in range(n_samples):
#         c = challenges[i]
#         transformed_challenges[i] = compute_tu_k(c)

#     return transformed_challenges


In [19]:
def compute_tu_k(c):
    k = len(c)
    tu_k = np.zeros(2*k)

    for i in range(k):
        tu_k[i] = 1 - 2*c[i]

    for i in range(k, 2*k-1):
        prod = 1
        for j in range(i-k, k):
            prod *= tu_k[j]
        tu_k[i] = prod
    tu_k[2*k-1] = tu_k[0]*tu_k[1]
    return tu_k
def my_map(challenges):
    n_samples = challenges.shape[0]
    k = challenges.shape[1]
    transformed_challenges = np.zeros((n_samples, 2*k))

    for i in range(n_samples):
        c = challenges[i]
        transformed_challenges[i] = compute_tu_k(c)

    return transformed_challenges

In [20]:
# X_train, X_valid = train_test_split(X, test_size=0.2, random_state=42)

In [21]:
X_train = my_map(X[:, :-2])
y_train0 = X[:, -2]
y_train1 = X[:, -1]

In [22]:
df = pd.DataFrame(X_train)
df


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,...,-1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0
1,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0
2,-1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,...,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0
3,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,...,1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,1.0
4,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,...,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0,...,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0
39996,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,...,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,1.0
39997,-1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,1.0
39998,1.0,1.0,1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,...,1.0,1.0,1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,1.0


In [28]:
param_grid = {
    'penalty': ['l1', 'l2'],  # Regularization penalty
    'C': [ 1.0, 5, 10],  # Inverse of regularization strength
    'solver': ['liblinear', 'saga']  # Optimization algorithm
}
grid_search = GridSearchCV(estimator=LogisticRegression(max_iter=5000), param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train0)
results = grid_search.cv_results_
print("Results for each parameter combination:")
for mean_test_score, std_test_score, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
    print(f"Mean Test Accuracy: {mean_test_score:.3f}, Std Test Accuracy: {std_test_score:.3f}, Params: {params}")

# Print the best parameters and best score
print("\nBest parameters found: ", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
# best_params = grid_search.best_params_
# print("Best Parameters:", best_params)
# print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Results for each parameter combination:
Mean Test Accuracy: 0.978, Std Test Accuracy: 0.001, Params: {'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear'}
Mean Test Accuracy: 0.978, Std Test Accuracy: 0.001, Params: {'C': 1.0, 'penalty': 'l1', 'solver': 'saga'}
Mean Test Accuracy: 0.978, Std Test Accuracy: 0.001, Params: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
Mean Test Accuracy: 0.978, Std Test Accuracy: 0.001, Params: {'C': 1.0, 'penalty': 'l2', 'solver': 'saga'}
Mean Test Accuracy: 0.978, Std Test Accuracy: 0.001, Params: {'C': 5, 'penalty': 'l1', 'solver': 'liblinear'}
Mean Test Accuracy: 0.978, Std Test Accuracy: 0.001, Params: {'C': 5, 'penalty': 'l1', 'solver': 'saga'}
Mean Test Accuracy: 0.978, Std Test Accuracy: 0.001, Params: {'C': 5, 'penalty': 'l2', 'solver': 'liblinear'}
Mean Test Accuracy: 0.978, Std Test Accuracy: 0.001, Params: {'C': 5, 'penalty': 'l2', 'solver': 'saga'}
Mean Test Accuracy: 0.978, Std 

In [ ]:
results = grid_search.cv_results_
results


{'mean_fit_time': array([ 11.60405631,  40.61479683,   4.06926742,  12.16965814,
         23.15818086,  74.09678774,   6.06025186,  22.21794643,
         51.63821888, 161.68721232,   8.2463253 ,  62.28411417,
         58.80851221, 189.01410079,   8.47482548,  84.77079315]),
 'std_fit_time': array([0.59887048, 0.41905238, 0.48754527, 0.47399653, 1.05234324,
        1.30611332, 0.77970773, 0.92591943, 4.8275994 , 3.47518013,
        0.6744765 , 1.91110091, 3.71577795, 4.31985826, 0.76437321,
        2.0513282 ]),
 'mean_score_time': array([0.01425743, 0.00957522, 0.01132641, 0.0127337 , 0.01278539,
        0.01178517, 0.01178074, 0.01022739, 0.01314926, 0.01529975,
        0.0122952 , 0.01223764, 0.01709313, 0.01875949, 0.01263695,
        0.01125832]),
 'std_score_time': array([0.00627427, 0.00016519, 0.00343534, 0.00391359, 0.0011205 ,
        0.00194476, 0.00399324, 0.00085602, 0.00354312, 0.00541048,
        0.00405269, 0.00251477, 0.00881013, 0.00836334, 0.0034652 ,
        0.000626

In [23]:
param_grid = {
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10],  # Regularization parameter
    'loss': ['hinge', 'squared_hinge'],  # Loss function
    'max_iter': [5000],  # Maximum number of iterations
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=LinearSVC(), param_grid=param_grid, cv=5, verbose=3, n_jobs=-1)

# Fit the GridSearchCV object
grid_search.fit(X_train, y_train1)

# Print training accuracy for each combination of parameters
results = grid_search.cv_results_
print("Results for each parameter combination:")
for mean_test_score, std_test_score, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
    print(f"Mean Test Accuracy: {mean_test_score:.3f}, Std Test Accuracy: {std_test_score:.3f}, Params: {params}")

# Print the best parameters and best score
print("\nBest parameters found: ", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Results for each parameter combination:
Mean Test Accuracy: 0.983, Std Test Accuracy: 0.001, Params: {'C': 0.0001, 'loss': 'hinge', 'max_iter': 5000}
Mean Test Accuracy: 0.989, Std Test Accuracy: 0.002, Params: {'C': 0.0001, 'loss': 'squared_hinge', 'max_iter': 5000}
Mean Test Accuracy: 0.989, Std Test Accuracy: 0.002, Params: {'C': 0.001, 'loss': 'hinge', 'max_iter': 5000}
Mean Test Accuracy: 0.993, Std Test Accuracy: 0.001, Params: {'C': 0.001, 'loss': 'squared_hinge', 'max_iter': 5000}
Mean Test Accuracy: 0.993, Std Test Accuracy: 0.001, Params: {'C': 0.01, 'loss': 'hinge', 'max_iter': 5000}
Mean Test Accuracy: 0.996, Std Test Accuracy: 0.001, Params: {'C': 0.01, 'loss': 'squared_hinge', 'max_iter': 5000}
Mean Test Accuracy: 0.995, Std Test Accuracy: 0.000, Params: {'C': 0.1, 'loss': 'hinge', 'max_iter': 5000}
Mean Test Accuracy: 0.997, Std Test Accuracy: 0.001, Params: {'C': 0.1, 'loss': 'squared_hinge', 'max_iter': 5000}

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [47]:
param_grid = {
    'C': [10],  # Regularization parameter
    'loss': ['hinge'],  # Loss function
    'max_iter': [10000],# Maximum number of iterations
    'tol': [0.000001, 0.00001, 0.00015, 0.001],
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=LinearSVC(), param_grid=param_grid, cv=5, verbose=3, n_jobs=-1)

# Fit the GridSearchCV object
grid_search.fit(X_train, y_train0)

# Print training accuracy for each combination of parameters
results = grid_search.cv_results_
print("Results for each parameter combination:")
for mean_test_score, std_test_score, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
    print(f"Mean Test Accuracy: {mean_test_score:.3f}, Std Test Accuracy: {std_test_score:.3f}, Params: {params}")

# Print the best parameters and best score
print("\nBest parameters found: ", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Results for each parameter combination:
Mean Test Accuracy: 0.981, Std Test Accuracy: 0.001, Params: {'C': 10, 'loss': 'hinge', 'max_iter': 10000, 'tol': 1e-06}
Mean Test Accuracy: 0.981, Std Test Accuracy: 0.001, Params: {'C': 10, 'loss': 'hinge', 'max_iter': 10000, 'tol': 1e-05}
Mean Test Accuracy: 0.981, Std Test Accuracy: 0.001, Params: {'C': 10, 'loss': 'hinge', 'max_iter': 10000, 'tol': 0.00015}
Mean Test Accuracy: 0.981, Std Test Accuracy: 0.001, Params: {'C': 10, 'loss': 'hinge', 'max_iter': 10000, 'tol': 0.001}

Best parameters found:  {'C': 10, 'loss': 'hinge', 'max_iter': 10000, 'tol': 1e-05}
Best cross-validation score: 0.98


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
param_grid = {
    'alpha': [0.1, 1.0, 10.0],  # Regularization parameter
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']  # Solver for optimization
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=RidgeClassifier(), param_grid=param_grid, cv=5, verbose=6, n_jobs=-1)

# Fit the GridSearchCV object

grid_search.fit(X_train, y_train0)

# Print training accuracy for each combination of parameters
results = grid_search.cv_results_
print("Results for each parameter combination:")
for mean_test_score, std_test_score, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
    print(f"Mean Test Accuracy: {mean_test_score:.3f}, Std Test Accuracy: {std_test_score:.3f}, Params: {params}")

# Print the best parameters and best score
print("\nBest parameters found: ", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))


Fitting 5 folds for each of 21 candidates, totalling 105 fits
Results for each parameter combination:
Mean Test Accuracy: 0.637, Std Test Accuracy: 0.002, Params: {'alpha': 0.1, 'solver': 'auto'}
Mean Test Accuracy: 0.637, Std Test Accuracy: 0.002, Params: {'alpha': 0.1, 'solver': 'svd'}
Mean Test Accuracy: 0.637, Std Test Accuracy: 0.002, Params: {'alpha': 0.1, 'solver': 'cholesky'}
Mean Test Accuracy: 0.637, Std Test Accuracy: 0.002, Params: {'alpha': 0.1, 'solver': 'lsqr'}
Mean Test Accuracy: 0.637, Std Test Accuracy: 0.002, Params: {'alpha': 0.1, 'solver': 'sparse_cg'}
Mean Test Accuracy: 0.637, Std Test Accuracy: 0.002, Params: {'alpha': 0.1, 'solver': 'sag'}
Mean Test Accuracy: 0.637, Std Test Accuracy: 0.002, Params: {'alpha': 0.1, 'solver': 'saga'}
Mean Test Accuracy: 0.637, Std Test Accuracy: 0.002, Params: {'alpha': 1.0, 'solver': 'auto'}
Mean Test Accuracy: 0.637, Std Test Accuracy: 0.002, Params: {'alpha': 1.0, 'solver': 'svd'}
Mean Test Accuracy: 0.637, Std Test Accuracy: 

In [57]:
def my_fit(Z_trn):
    # Extract features and responses from Z_trn
    X_train = Z_trn[:, :-2]
    y_train0 = Z_trn[:, -2]
    y_train1 = Z_trn[:, -1]

    # Map the training data
    X_train_mapped = my_map(X_train)
    # 'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'
    # 'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'
    # Fit two linear models for Response0 (y_train0) and Response1 (y_train1)
    # model0 = LogisticRegression(solver='liblinear', C=0.1, penalty='l1', max_iter=1000)
    # model1 = LogisticRegression(solver='liblinear', C=0.1, penalty='l1', max_iter=1000)
    model0 = LinearSVC( C=10, loss = "hinge", max_iter=10000, tol=0.00015)
    model1 = LinearSVC( C=10, loss = "squared_hinge", max_iter=10000, tol=0.00015)
    # Fit the models
    model0.fit(X_train_mapped, y_train0)
    model1.fit(X_train_mapped, y_train1)

    # Extract coefficients and intercepts
    W0, b0 = model0.coef_, model0.intercept_
    W1, b1 = model1.coef_, model1.intercept_
    # print(W0, b0)
    # print(W1, b1)
    # Predict training labels to calculate training accuracy
    y_pred0 = model0.predict(X_train_mapped)
    y_pred1 = model1.predict(X_train_mapped)

    # Calculate training accuracy
    acc0 = accuracy_score(y_train0, y_pred0)
    acc1 = accuracy_score(y_train1, y_pred1)


    return W0, b0, W1, b1, acc0, acc1

In [58]:
n_trials = 1
d_size = 0
t_train = 0
t_map = 0
acc0_test = 0
acc1_test = 0
acc0_train = 0
acc1_train = 0


In [59]:
for t in range(n_trials):
    tic = tm.perf_counter()
    w0, b0, w1, b1, acc0_, acc1_ = my_fit(X)
    toc = tm.perf_counter()

    t_train += toc - tic
    w0 = w0.reshape(-1)
    w1 = w1.reshape(-1)

    d_size += max(w0.shape[0], w1.shape[0])

    tic = tm.perf_counter()
    feat = my_map(X_test[:, :-2])
    toc = tm.perf_counter()
    t_map += toc - tic


    scores0 = feat.dot(w0) + b0
    scores1 = feat.dot(w1) + b1

    pred0 = np.zeros_like(scores0)
    pred0[scores0 > 0] = 1
    pred1 = np.zeros_like(scores1)
    pred1[scores1 > 0] = 1

    acc0_test += np.average(X_test[:, -2] == pred0)
    acc1_test += np.average(X_test[:, -1] == pred1)

    acc0_train += acc0_
    acc1_train += acc1_

d_size /= n_trials
t_train /= n_trials
t_map /= n_trials
acc0_test /= n_trials
acc1_test /= n_trials
acc0_train /= n_trials
acc1_train /= n_trials

print(f"Avg Training Time: {t_train:.4f} seconds")
print(f"Avg Mapping Time: {t_map:.4f} seconds")
print(f"Avg Training Accuracy for Response0: {acc0_train:.4f}")
print(f"Avg Training Accuracy for Response1: {acc1_train:.4f}")
print(f"Avg Accuracy for Response0: {acc0_test:.4f}")
print(f"Avg Accuracy for Response1: {acc1_test:.4f}")
print(f"Avg dimension Size: {d_size:.4f}")

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Avg Training Time: 16.1424 seconds
Avg Mapping Time: 1.5748 seconds
Avg Training Accuracy for Response0: 0.9818
Avg Training Accuracy for Response1: 0.9994
Avg Accuracy for Response0: 0.9842
Avg Accuracy for Response1: 0.9989
Avg dimension Size: 64.0000
